In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.models import Sequential

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

%matplotlib inline

Using TensorFlow backend.


In [2]:
LABELS = 10 # Number of different types of labels (1-10)
WIDTH = 28 # width / height of the image
CHANNELS = 1 # Number of colors in the image (greyscale)
VALID = 10000 # Validation data size

In [3]:
data = pd.read_csv('../data/digit-recognizer/train.csv')

In [4]:
data.shape

(42000, 785)

In [5]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Prepare Data

        - Apply one hot encoding on labels
        - Convert the data into shape of (#rows, height, width, channels) shape
        - Normalize the input data (TODO)
        
        
We will check two approaches:
        - Feature Extraction
        - Fine tuning
    

In [6]:
labels = np.array(data.pop('label'))
labels = LabelEncoder().fit_transform(labels)[:, None]
labels = OneHotEncoder().fit_transform(labels).todense()

In [8]:
data = data/255.0
data = data.values.reshape(-1,28,28)

In [9]:
data.shape

(42000, 28, 28)

In [10]:
data = tf.pad(data, [[0, 0], [2,2], [2,2]])

In [11]:
data.shape

TensorShape([42000, 32, 32])

In [12]:
data = tf.expand_dims(data, axis=3, name=None)
data = tf.repeat(data, 3, axis=3)


In [13]:
data.shape

TensorShape([42000, 32, 32, 3])

In [14]:
train_data, valid_data = data[:-VALID], data[-VALID:]
train_labels, valid_labels = labels[:-VALID], labels[-VALID:]

## MobileNet Model

In [18]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [31]:
def pretrain_mobilenet(img_shape):
    base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,
                       include_top=False,
                       weights='imagenet')

    # freeze the base model by making it non trainable
    base_model.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=img_shape) 

    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(inputs)

    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 

    # use global avg pooling to summarize the info in each channel
    x = tf.keras.layers.GlobalAveragePooling2D()(x) 

    # include dropout with probability of 0.2 to avoid overfitting
    x = tf.keras.layers.Dropout(0.2)(x)

    # use a prediction layer 
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model
    

In [32]:
base_model = pretrain_mobilenet((32, 32, 3))

In [33]:
base_learning_rate = 0.001
base_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [36]:
history = base_model.fit(train_data, [train_labels, train_labels, train_labels], 
                    validation_data=(valid_data, [valid_labels, valid_labels, valid_labels]), 
                    batch_size=64, epochs=5)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), for inputs ['dense_2'] but instead got the following list of 3 arrays: [matrix([[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., ...

In [35]:
initial_epochs = 5
history = base_model.fit(train_data, validation_data=valid_data, epochs=initial_epochs)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [22]:
valid_data.shape

TensorShape([10000, 32, 32, 3])

In [26]:
base_model.trainable = False
image_var = tf.Variable(image_batch)
pred = base_model(image_var)

NameError: name 'image_batch' is not defined

In [30]:
pred = base_model(valid_data)

ValueError: in converted code:

    /Users/nehapawar/Desktop/neha/venv3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:2545 _defun_call  *
        return self._make_op(inputs)
    /Users/nehapawar/Desktop/neha/venv3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:2523 _make_op
        c_op = ops._create_c_op(graph, node_def, inputs, control_inputs=[])
    /Users/nehapawar/Desktop/neha/venv3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1622 _create_c_op
        raise ValueError(str(e))

    ValueError: 2 errors while building NodeDef 'truediv_1' using Op<name=RealDiv; signature=x:T, y:T -> z:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_UINT8, DT_INT8, DT_UINT16, DT_INT16, DT_INT32, DT_INT64, DT_COMPLEX64, DT_COMPLEX128]>:
    Inconsistent values for attr 'T' DT_DOUBLE vs. DT_FLOAT
    Inconsistent values for attr 'T' DT_DOUBLE vs. DT_FLOAT
